In [304]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv


In [305]:
from numpy import array, argmax, random, take
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, RepeatVector, TimeDistributed,Masking
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
pd.set_option('display.max_colwidth', 200)

In [306]:
df=pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv")
df

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...,...
127602,indic2012,Examples of art deco construction can be found around Marine Drive and Oval Ground areas.,आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल मैदान के किनारे दिखाई देते हैं।
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।
127604,tides,"As for the other derivatives of sulphur , the country 's needs of iron , copper , sodium , etc . sulphates were limited , and the production achieved after the war was generally adequate .","जहां तक गंधक के अन्य उत्पादों का प्रश्न है , देश में लोहे , तांबे , सोडियम , सल्फेट आदि की आवश्यकता सीमित थी और युद्धोपरांत हुआ उत्पादन सामान्य रूप से पर्याप्त था ."
127605,tides,its complicated functioning is defined thus in a popular riddle :,Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .


In [307]:
df.isnull().sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [308]:
df.shape

(127607, 3)

In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127607 entries, 0 to 127606
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   source            127607 non-null  object
 1   english_sentence  127605 non-null  object
 2   hindi_sentence    127607 non-null  object
dtypes: object(3)
memory usage: 2.9+ MB


In [310]:
df.describe()

,source,english_sentence,hindi_sentence
count,127607,127605,127607
unique,3,124317,97662
top,tides,(Laughter),(हँसी)
freq,50000,555,212


# As we see that this dataset is very big so let us take 40000 rows from this dataset..

In [311]:
df.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,"The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .","कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी ."
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
9,tides,You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .,"हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . ."


In [312]:
new_df=df.sample(n = 40000,random_state=42) 
new_df

,source,english_sentence,hindi_sentence
3556,tides,He declares the result and reports it to the Election Commission and to the Secretary-General of the concerned House .,वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है .
25899,ted,was a little uncomfortable for them.,थोडा कठिन था।
89038,indic2012,But Mulla Assamudin was proved to be not eligible.,मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए।
78212,ted,I would never have to make a book and then present it to a gallery,मुझे कभी भी किताब बना कर किसी प्रदर्शन-स्थल को देने की ज़रुरत नहीं पड़ी
96955,indic2012,headind kaun banega crorepati,शीर्षक कौन बनेगा करोड़पति (Kaun Banega Crorepati).
...,...,...,...
74752,tides,"The party will seek to unite all radical organisations under a national executive so that there is Subhas Chandra expressed the view that there were , several reasons why Communism would not be ad...","पार्टी को सारे परिवर्तनकामी संगठनों को एक राष्ट्रीय कार्यकारिणी के अंतर्गत एकसूत्र करने का प्रयास करना होगा , ताकि राष्ट्रीय जीवन में कई मोर्चों पर एक साथ मिली-जुली कोशिशें हों ."
24388,ted,Does it mean the problem is over? The answer is no.,क्या इसका ये मतलब है कि यह समस्या ख़त्म हो गई है? इस सवाल का जवाब नहीं है।
6039,tides,His heart was restless “ like a serpent reviving from its long winter 's sleep . ”,"उसका चित्त विचलित हो उठता है , उस सर्प की तरह जो शीतकाल में अपनी लंबी नींद के बाद जग उठता है ."
109035,ted,On one side,एक तरफ तो


In [313]:
new_df=new_df.reset_index()

In [314]:
new_df.head()

,index,source,english_sentence,hindi_sentence
0,3556,tides,He declares the result and reports it to the Election Commission and to the Secretary-General of the concerned House .,वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है .
1,25899,ted,was a little uncomfortable for them.,थोडा कठिन था।
2,89038,indic2012,But Mulla Assamudin was proved to be not eligible.,मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए।
3,78212,ted,I would never have to make a book and then present it to a gallery,मुझे कभी भी किताब बना कर किसी प्रदर्शन-स्थल को देने की ज़रुरत नहीं पड़ी
4,96955,indic2012,headind kaun banega crorepati,शीर्षक कौन बनेगा करोड़पति (Kaun Banega Crorepati).


In [315]:
new_df['english_sentence'][11]

'What if I need general information about applying for a work permit ?'

In [316]:
new_df.shape

(40000, 4)

In [317]:
new_df.dtypes

index                int64
source              object
english_sentence    object
hindi_sentence      object
dtype: object

# Now cleaning the sentences..

In [318]:
new_df['english_sentence'] = new_df['english_sentence'].astype(str)

In [319]:
# Lowercase all characters
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: x.lower())
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: x.lower())

In [320]:
import  re
import string
from string import digits

In [321]:
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: re.sub("'", '', x))
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [322]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: x.translate(remove_digits))
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

new_df['hindi_sentence'] = new_df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: x.strip())
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: x.strip())
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

# Adding START and END tokens to the target sequences(i.e Hindi Sentences):- 

In [323]:
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x:'START_ ' + x + ' _END') 

In [324]:
new_df.head()

,index,source,english_sentence,hindi_sentence
0,3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END
3,78212,ted,i would never have to make a book and then present it to a gallery,START_ मुझे कभी भी किताब बना कर किसी प्रदर्शनस्थल को देने की ज़रुरत नहीं पड़ी _END
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END


# Now calculating the english and hindi sentence lengths..

In [325]:
senlen1=[]

In [326]:
for i in new_df['english_sentence']:
    res = len(i.split()) 
    senlen1.append(res)

In [327]:
# senlen1

In [328]:
new_df['engsentlen']=senlen1

In [329]:
new_df.head()

,index,source,english_sentence,hindi_sentence,engsentlen
0,3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END,19
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END,6
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END,9
3,78212,ted,i would never have to make a book and then present it to a gallery,START_ मुझे कभी भी किताब बना कर किसी प्रदर्शनस्थल को देने की ज़रुरत नहीं पड़ी _END,15
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END,4


In [330]:
len(new_df['english_sentence'])

40000

In [331]:
senlen2=[]

In [332]:
for j in new_df['hindi_sentence']:
     res = len(j.split())
     senlen2.append(res)

In [333]:
# senlen2

In [334]:
new_df['hinsenlen']=senlen2

In [335]:
new_df.head(10)

,index,source,english_sentence,hindi_sentence,engsentlen,hinsenlen
0,3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END,19,22
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END,6,5
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END,9,8
3,78212,ted,i would never have to make a book and then present it to a gallery,START_ मुझे कभी भी किताब बना कर किसी प्रदर्शनस्थल को देने की ज़रुरत नहीं पड़ी _END,15,16
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END,4,9
5,317,tides,no other national leader except nehru shared that concern,START_ नेहरू को छोड़कर और किसी भी राष्ट्र नेता के मन में यह चिंता नहीं थे _END,9,17
6,109185,indic2012,bachchan continued togive their support to the samajvadi party which included political propaganda and promotions,START_ बच्चन ने समाजवादी पार्टी के लिए अपना समर्थन देना जारी रखा जिसमें राजनैतिक अभियान अर्थात प्रचार प्रसार करना शामिल था। _END,15,22
7,117804,tides,innocent people were shot arrested jailed and tortured by the british rulers,START_ अंग्रेज शासकों ने निर्दोष लोगों की जान ली उन्हें पकड़कर जेलों में बंद किया और यातनाएं दीं _END,12,19
8,126165,tides,it means that fascism will make a triumphant attempt at world dominion in cooperation with the fascism of japan,START_ इसका मतलब है कि फासिस्टवाद जापान के फासिस्टवाद के साथ मिलकर सारी दुनिया पर कामयाबी के साथ कब्जा करने की कोशिश करेगा _END,19,24
9,37263,ted,just as the poets and writers described,START_ जैसे कवियों और लेखकों ने वर्णन किया है _END,7,10


Our dataset is now cleaned..

In [336]:
new_df=new_df[new_df['engsentlen']<=10]
new_df=new_df[new_df['hinsenlen']<=10]

In [337]:
new_df.head()

,index,source,english_sentence,hindi_sentence,engsentlen,hinsenlen
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END,6,5
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END,9,8
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END,4,9
9,37263,ted,just as the poets and writers described,START_ जैसे कवियों और लेखकों ने वर्णन किया है _END,7,10
10,8579,tides,the rich flora of central america is under threat,START_ मध्य अमेरिकी बहुमूल्य वनस्पति खतरे में है _END,9,9


In [338]:
new_df.shape

(11981, 6)

In [339]:
print("Maximum length of English Sentence ",max(new_df['engsentlen']))
print("Maximum length of Hindi Sentence ",max(new_df['hinsenlen']))

Maximum length of English Sentence  10
Maximum length of Hindi Sentence  10


In [340]:
max_length_eng=max(new_df['engsentlen'])
max_length_hin=max(new_df['hinsenlen'])

In [341]:
def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [342]:
english_tokenizer = tokenization(new_df['english_sentence'].values)

# Creating english and hindi vocabulary:-

In [343]:
english_words=set()
for eng in new_df['english_sentence']:
    for word in eng.split():
        if word not in english_words:
            english_words.add(word)

hindi_words=set()
for hin in new_df['hindi_sentence']:
    for word in hin.split():
        if word not in hindi_words:
            hindi_words.add(word)

In [344]:
english_words.add(' ')

In [345]:
hindi_words.add(' ')

In [346]:
len(english_words)

9931

In [347]:
len(hindi_words)

10720

In [348]:
input_texts = []
target_texts = []

# Making the seperate lists of English and Hindi Sentences:-

In [349]:
for i in new_df['english_sentence']:
    input_texts.append(i)

In [350]:
# input_texts

In [351]:
len(input_texts)

11981

In [352]:
for i in new_df['hindi_sentence']:
    target_texts.append(i)

In [353]:
# target_texts

In [354]:
num_encoder_tokens = len(english_words)
num_decoder_tokens = len(hindi_words)

# Created a Dictionary to convert a all the words to integer:-

In [355]:
input_words = sorted(list(english_words))
target_words = sorted(list(hindi_words))


input_token_index = dict(
    [(char, i) for i, char in enumerate(input_words)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_words)])

In [356]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [357]:
data=np.zeros((2,3),dtype='float32')
data

array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [358]:
data=np.zeros((2,2,4),dtype='float32')
data

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.]]], dtype=float32)

In [359]:
data=np.zeros((2,10,6),dtype='float32')
data

array([[[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]], dtype=float32)

# So According to this we will encode the data as input by converting it into 3D Numpy Array. So according to our observations we will set 1st parameter as the total input data, 2nd as the max length among all the sentences and the 3rd as the dimension as each word will be represented by a number between 0 to the last index of word present..

In [360]:
encoder_input_data = np.zeros(
    (len(input_texts), max_length_eng, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_length_hin, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts),  max_length_hin, num_decoder_tokens),
    dtype='float32')

In [361]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

# encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences.

In [362]:
encoder_input_data.shape

(11981, 10, 9931)

# decoder_input_data is a 3D array of shape (num_pairs, max_hindi_sentence_length, num_hindi_characters) containg a one-hot vectorization of the Hindi sentences.

In [363]:
decoder_input_data.shape

(11981, 10, 10720)

# decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].

In [364]:
decoder_target_data.shape

(11981, 10, 10720)

In [365]:
text='This is Rahul'

In [366]:
for word in text.split():
    print(word)

This
is
Rahul


In [367]:
# input_token_index


In [368]:
input_token_index['poets']

6547

In [369]:
len(input_token_index)

9931

# Doing One Hot Encoding of All the Three 3D Arrays:-

In [370]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Loop over each char in an input text
        for t, word in enumerate(input_text.split()):
                    #print(word,t)
                    encoder_input_data[i, t, input_token_index[word]]=1. #input token index gives index from the dictionary created
                    
        encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.            
        for t, word in enumerate(target_text.split()):
#                     print(t,word)
                    if t<len(target_text.split())-1: # We are subtracting 1 as we are not taking the START Keyword at the starting.
                        decoder_input_data[i, t,target_token_index[word]]=1. #input token index gives index from the dictionary created of target sequence
                   
                    if t>0: 
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep So we will not include t=0th word. We will take words after that.
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
                        
        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
        decoder_target_data[i, t:, target_token_index[' ']] = 1.

# Encoder Decoder Architecture:-

In [371]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [372]:
encoder_inputs=Input(shape=(None, num_encoder_tokens))
encoder=LSTM(128,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#here we will ignore the encoder outputs and take only the two states.
encoder_states=[state_h,state_c]

In [373]:
# Setting up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We are setting up our decoder to return full output sequences,
# and to return internal states as well. We are not using the 
# return states in the training model, but we will use them in inference.

decoder = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Defining the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# decoder inputs are the real input that we are giving the as input to each LSTM unit and the outputs are predicted by the LSTM.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Model Compilation and Training:-

In [377]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [378]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None, 9931)   0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None, 10720)  0                                            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 128), (None, 5150720     input_13[0][0]                   
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 128),  5554688     input_14[0][0]                   
                                                                 lstm_9[0][1]               

In [380]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=128,epochs=20,validation_split=0.2)

Train on 9584 samples, validate on 2397 samples
Epoch 1/20
9584/9584 [==============================] - 25s 3ms/step - loss: 3.2883 - accuracy: 0.5204 - val_loss: 3.9722 - val_accuracy: 0.4776
Epoch 2/20
9584/9584 [==============================] - 25s 3ms/step - loss: 3.2504 - accuracy: 0.5251 - val_loss: 3.8961 - val_accuracy: 0.4846
Epoch 3/20
9584/9584 [==============================] - 26s 3ms/step - loss: 3.2141 - accuracy: 0.5272 - val_loss: 3.8620 - val_accuracy: 0.4864
Epoch 4/20
9584/9584 [==============================] - 25s 3ms/step - loss: 3.1741 - accuracy: 0.5321 - val_loss: 3.8506 - val_accuracy: 0.4870
Epoch 5/20
9584/9584 [==============================] - 25s 3ms/step - loss: 3.1333 - accuracy: 0.5357 - val_loss: 3.8842 - val_accuracy: 0.4909
Epoch 6/20
9584/9584 [==============================] - 25s 3ms/step - loss: 3.0938 - accuracy: 0.5393 - val_loss: 3.8284 - val_accuracy: 0.4873
Epoch 7/20
9584/9584 [==============================] - 25s 3ms/step - loss: 3.053

# Saving the model:-

In [381]:
model.save('final_model')

# Inference Model:-

In [385]:
model

In [382]:
encoder_model = Model(encoder_inputs, encoder_states)
# The encoder model which takes the encoder data as inputs and returns the hidden states of the input sequence..
# So we can use the encoder model to get the hidden states of the input sequence..
# Model(input,output)
# Input a sequence and output states.
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# Here we will be using the hidden states
decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
# In the inference state we are trying to predict the next word using the previous state output. So its architecture is like this.
#In this step we are using the decoder model hidden states.
#This model is taking the input of a sequence with hidden states and give output of token with hidden states so that that can be used to 
# predict new words.
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [386]:
encoder_model

In [387]:
decoder_model

# So here we are using the decoder model that is same as the trained model..

# Predict Sentences:-

In [383]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['START_']] = 1.
    

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
       
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
       
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence = decoded_sentence + ' '+ sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > max_length_hin):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [384]:
for seq_index in range(400,500):
   # Taking input from the training set
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence in English:', input_texts[seq_index])
    print('Translated sentence in Hindi:', decoded_sentence)

-
Input sentence in English: but this is very shaky
Translated sentence in Hindi:  लेकिन यह बहुत
-
Input sentence in English: the best teacher according to tagore is nature
Translated sentence in Hindi:  वे के लिए भी
-
Input sentence in English: which is a great triumph
Translated sentence in Hindi:  यह एक एक है
-
Input sentence in English: relatively sparse number of producers
Translated sentence in Hindi:  मुंबई की राजधानी
-
Input sentence in English: at the end of the week
Translated sentence in Hindi:  एक सालों के
-
Input sentence in English: lankakand yudhhakand
Translated sentence in Hindi:  दाल _END
-
Input sentence in English: subhashchandra was their th child and th son
Translated sentence in Hindi:  इनके सरकार
-
Input sentence in English: like our economic and political worlds
Translated sentence in Hindi:  और और आर्थिक
-
Input sentence in English: state judiciary
Translated sentence in Hindi:  वाह्य सूत्र
-
Input sentence in English: agra was the capital of moghul empire fro

-
Input sentence in English: at models that span years
Translated sentence in Hindi:  एक एक अलग होने
-
Input sentence in English: this was the booklet for this organization
Translated sentence in Hindi:  यह एक नाम का
-
Input sentence in English: mysore pak
Translated sentence in Hindi:  फतेहपुर _END
-
Input sentence in English: to doodle officially means
Translated sentence in Hindi:  इसकी से क्या
